In [2]:
import pandas as pd
import numpy as np
#from shapely import wkt
import geopandas as gpd

# Zoning Data

In [3]:
multifamily = {'R-M', 'R-M(PD)', 'R-M(CL)', 'R-2', 'R-2(PD)', 'UV', 'CP', 
               'CP(PD)', 'MUC', 'MUN', 'DC', 'DC-NT1', 'DC(PD)'}
#confirm what R-M(CL) means
#make sure we don't need to include others

In [4]:
sj_zips = gpd.read_file('Zip_Code_Boundary.geojson')
#sj_zips['geometry'] = sj_zips['geometry'].apply(wkt.loads)
#sj_zips = sj_zips.to_crs(epsg = 26910) #converting crs to planar

sj_zones = gpd.read_file('zoning_data/Zoning_2022.geojson')
#sj_zones['geometry'] = sj_zones['geometry'].apply(wkt.loads)

#sj_zones = sj_zones.to_crs(epsg = 26910) #converting crs to planar


In [40]:
#sj_zones['area'] = sj_zones.area #creating area column

In [5]:
# def calculate_percentage(group):
#     total_area = group['area'].sum()
#     rm_area = group[group['ZONINGABBREV'].isin(multifamily)]['area'].sum()
#     percentage = rm_area / total_area
#     return percentage

def calculate_percentage(group):
    total = len(group)
    rm_rows = len(group[group['ZONINGABBREV'].isin(multifamily)])
    
    return rm_rows / total
    

In [43]:
def calc_area(group):
    return group['area'].sum()

**Final Year**

In [7]:
joined_data = gpd.sjoin(sj_zips, sj_zones, how='inner', predicate='intersects')


percentages = (
    joined_data.groupby('ZIPCODE').apply(calculate_percentage)
    .reset_index(name='multifam_percentage')
)

# areas = (
#     joined_data.groupby('ZIPCODE').apply(calc_area)
#     .reset_index(name='total_area')
# )

joined_data = joined_data.merge(percentages, on = 'ZIPCODE', how = 'left')
#joined_data = joined_data.merge(areas, on = 'ZIPCODE', how = 'left')


abbrev_data = joined_data[['ZIPCODE', 'FACILITYID', 'ZONINGABBREV', 'multifam_percentage']] #would add total area here
abbrev_data['ZIPCODE'] = abbrev_data['ZIPCODE'].astype('int64')

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_31348/2437229890.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbrev_data['ZIPCODE'] = abbrev_data['ZIPCODE'].astype('int64')


**Base Year (2012)**

In [8]:
base_zones = gpd.read_file('zoning_data/Zoning_2012.geojson')
base_zones = base_zones.to_crs(sj_zones.crs)

#base_zones['area'] = base_zones.area #creating area column


base_joined = gpd.sjoin(sj_zips, base_zones, how='inner', predicate='intersects')

base_percentages = (
    base_joined.groupby('ZIPCODE').apply(calculate_percentage)
    .reset_index(name='multifam_percentage_2012')
)


base_joined = base_joined.merge(base_percentages, on = 'ZIPCODE', how = 'left')


abbrev_base = base_joined[['ZIPCODE', 'FACILITYID', 'ZONINGABBREV', 'multifam_percentage_2012']]
abbrev_base['ZIPCODE'] = abbrev_base['ZIPCODE'].astype('int64')

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_31348/2271567065.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbrev_base['ZIPCODE'] = abbrev_base['ZIPCODE'].astype('int64')


**Intermediate Year (2015)**

In [9]:
zones_15 = gpd.read_file('zoning_data/Zoning_2015.geojson')

zones_15 = zones_15.to_crs(sj_zones.crs)
#zones_15['area'] = zones_15.area #creating area column


joined_15 = gpd.sjoin(sj_zips, zones_15, how='inner', predicate='intersects')

percentages_15 = (
    joined_15.groupby('ZIPCODE').apply(calculate_percentage)
    .reset_index(name='multifam_percentage_2015')
)



joined_15 = joined_15.merge(percentages_15, on = 'ZIPCODE', how = 'left')


abbrev_15 = joined_15[['ZIPCODE', 'FACILITYID', 'ZONINGABBREV', 'multifam_percentage_2015']] 
abbrev_15['ZIPCODE'] = abbrev_15['ZIPCODE'].astype('int64')

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_31348/3835774195.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbrev_15['ZIPCODE'] = abbrev_15['ZIPCODE'].astype('int64')


**Joining**

In [10]:
to_join_base = abbrev_base[['ZIPCODE', 'multifam_percentage_2012']].groupby('ZIPCODE').max()
to_join_15 = abbrev_15[['ZIPCODE', 'multifam_percentage_2015']].groupby('ZIPCODE').max()
to_join_final = abbrev_data[['ZIPCODE', 'multifam_percentage']].groupby('ZIPCODE').max() #would add total area

temp = to_join_base.merge(to_join_15, how = 'inner', left_index = True, right_index = True)
zip_panel = temp.merge(to_join_final, how = 'inner', left_index = True, right_index = True)

In [11]:
#creating independent variables
zip_panel['initial_diff'] = zip_panel['multifam_percentage_2015'] - zip_panel['multifam_percentage_2012']
zip_panel['final_diff'] = zip_panel['multifam_percentage'] - zip_panel['multifam_percentage_2015']

# Rent Data

In [12]:
#final year rents
rents = pd.read_excel('fy2021_safmrs_revised.xlsx')

rents.rename({
    'ZIP\nCode': 'ZIP Code', 
    'SAFMR\n0BR': 'SAFMR 0BR',
    'SAFMR\n0BR - 90%\nPayment\nStandard': 'SAFMR 0BR - 90% Payment Standard',
    'SAFMR\n0BR - 110%\nPayment\nStandard': 'SAFMR 0BR - 110% Payment Standard',
    'SAFMR\n1BR': 'SAFMR 1BR',
       'SAFMR\n1BR - 90%\nPayment\nStandard': 'SAFMR 1BR - 90% Payment Standard',
       'SAFMR\n1BR - 110%\nPayment\nStandard': 'SAFMR 1BR - 110% Payment Standard', 
    'SAFMR\n2BR': 'SAFMR 2BR',
       'SAFMR\n2BR - 90%\nPayment\nStandard': 'SAFMR 2BR - 90% Payment Standard',
       'SAFMR\n2BR - 110%\nPayment\nStandard': 'SAFMR 2BR - 110% Payment Standard', 
    'SAFMR\n3BR': 'SAFMR 3BR',
       'SAFMR\n3BR - 90%\nPayment\nStandard': 'SAFMR 3BR - 90% Payment Standard',
       'SAFMR\n3BR - 110%\nPayment\nStandard': 'SAFMR 3BR - 110% Payment Standard', 
    'SAFMR\n4BR': 'SAFMR 4BR',
       'SAFMR\n4BR - 90%\nPayment\nStandard': 'SAFMR 4BR - 90% Payment Standard',
       'SAFMR\n4BR - 110%\nPayment\nStandard': 'SAFMR 4BR - 110% Payment Standard'
}, axis = 1, inplace = True)

abbrev_rents = rents[['ZIP Code', 'SAFMR 0BR', 'SAFMR 1BR',
                     'SAFMR 2BR', 'SAFMR 3BR', 'SAFMR 4BR']]
del rents

In [13]:
#base year rents
rents_2012 = pd.read_excel('small_area_fmrs_fy2012.xls')

rents_2012 = rents_2012[rents_2012['County Name'] == 'Santa Clara County']
rents_2012 = rents_2012[['ZIP', 'area_rent_br0', 'area_rent_br1', 'area_rent_br2', 'area_rent_br3', 'area_rent_br4']]

rents_2012.rename({
    'area_rent_br0': '0BR_base', 'area_rent_br1': '1BR_base', 
    'area_rent_br2': '2BR_base', 'area_rent_br3': '3BR_base', 
    'area_rent_br4': '4BR_base'
}, axis = 1, inplace = True)

rents_2012['ZIP'] = rents_2012['ZIP'].astype('int64')

In [14]:
#adjusting for inflation
rents_2012['0BR_base'] = rents_2012['0BR_base'] * 1.1802
rents_2012['1BR_base'] = rents_2012['1BR_base'] * 1.1802
rents_2012['2BR_base'] = rents_2012['2BR_base'] * 1.1802
rents_2012['3BR_base'] = rents_2012['3BR_base'] * 1.1802
rents_2012['4BR_base'] = rents_2012['4BR_base'] * 1.1802

**Calculating change in rent**

In [15]:
merged_rents = rents_2012.merge(abbrev_rents, left_on = 'ZIP', right_on = 'ZIP Code')

## 'SAFMR xBR' is the convention used in 2021, compared to xBR_base being used in 2012
merged_rents['0BR_change'] = merged_rents['SAFMR 0BR'] - merged_rents['0BR_base']
merged_rents['1BR_change'] = merged_rents['SAFMR 1BR'] - merged_rents['1BR_base']
merged_rents['2BR_change'] = merged_rents['SAFMR 2BR'] - merged_rents['2BR_base']
merged_rents['3BR_change'] = merged_rents['SAFMR 3BR'] - merged_rents['4BR_base']
merged_rents['4BR_change'] = merged_rents['SAFMR 4BR'] - merged_rents['4BR_base']

changes = merged_rents[['ZIP', '0BR_change', '1BR_change', 
                        '2BR_change', '3BR_change', '4BR_change',
                       '0BR_base', '1BR_base', '2BR_base', '3BR_base', '4BR_base']]

# Demographic Data (controls)

In [16]:
test_demo = pd.read_csv('controls_data/full_2012_demographics.csv')

test_demo['Label (Grouping)'] = test_demo['Label (Grouping)'].apply(lambda x: str(x).replace(u'\xa0', u''))
test_demo.columns = test_demo.columns.str.replace('!!', ' ')


test_demo['Label (Grouping)'] = test_demo['Label (Grouping)'].apply(lambda x: str(x).replace(u'\xa0', u''))
test_demo.columns = test_demo.columns.str.replace('!!', ' ')


og = test_demo.copy()

#to get the rows outside of "one race" dropdown
rows = [2] + list(range(60, 70))
test_demo = test_demo.iloc[rows]


#note that median age and total housing units are in 'estimate' columns
vals_to_keep = {'Male', 'White', 'Black or African American',
       'American Indian and Alaska Native', 'Asian', 'Native Hawaiian and Other Pacific Islander',
                'Some Other Race', 'Hispanic or Latino (of any race)',
               }

test_demo = test_demo[test_demo['Label (Grouping)'].isin(vals_to_keep)]


test_demo = test_demo.filter(regex='^(?=.*(?i)percent|label)(?!.*(?i)percent margin of error)')

pivoted = test_demo.set_index('Label (Grouping)').T
pivoted.reset_index(inplace = True)
pivoted.rename({'index': 'ZIP'}, axis = 1, inplace = True)
pivoted['ZIP'] = pivoted['ZIP'].str.extract(r'ZCTA5 (\d+) Percent')[0]

In [17]:
# Getting the non-percent values
og = og.filter(regex='^(?=.*(?i)estimate|label)(?!.*(?i)estimate margin of error)')

og = og.set_index('Label (Grouping)')
og_pivot = og[(og.index == 'Total housing units') | (og.index == 'Median age (years)') | 
              (og.index == 'Total population')].T

og_pivot = og_pivot.T.drop_duplicates(keep = 'first').T

og_pivot.reset_index(inplace = True)
og_pivot.rename({'index': 'ZIP'}, axis = 1, inplace = True)
og_pivot['ZIP'] = og_pivot['ZIP'].str.extract(r'ZCTA5 (\d+) Estimate')[0]

no_income = pivoted.merge(og_pivot, on = 'ZIP')

In [18]:
#2012 income data

raw_med_12 = pd.read_csv('controls_data/sj_med_income_2012.csv')
raw_med_12.columns = raw_med_12.columns.str.replace('!!', ' ')

mean_med = raw_med_12[(raw_med_12['Label (Grouping)'] == 'Median income (dollars)') 
           | (raw_med_12['Label (Grouping)'] == 'Mean income (dollars)')]

mean_med = mean_med.filter(regex=r'^(ZCTA5 \d{5} Households Estimate|Label \(Grouping\))$')


pivoted_income = mean_med.set_index('Label (Grouping)').T


pivoted_income.reset_index(inplace = True)
pivoted_income.rename({'index': 'ZIP'}, axis = 1, inplace = True)
pivoted_income['ZIP'] = pivoted_income['ZIP'].str.extract(r'ZCTA5 (\d+) Households Estimate')[0]
pivoted_income.replace({'-': np.nan}, inplace = True)

pivoted_income['Median income (dollars)'] = (
    pivoted_income['Median income (dollars)'].str.replace(',', '').astype('float64')
)

pivoted_income['Mean income (dollars)'] = (
    pivoted_income['Mean income (dollars)'].str.replace(',', '').astype('float64')
)

full_controls = no_income.merge(pivoted_income, on = 'ZIP')

In [19]:
#cleaning

full_controls.replace('-', np.nan, inplace = True)

def clean_pct(column):
    return column.str.rstrip('%').astype(float) / 100

def clean_totals(val):
    if isinstance(val, str):
        return float(val.replace(',', ''))
    else:
        return val

to_clean = ['Male', 'White', 'Black or African American', 'American Indian and Alaska Native',
           'Asian', 'Native Hawaiian and Other Pacific Islander', 'Hispanic or Latino (of any race)']

full_controls[to_clean] = full_controls[to_clean].apply(clean_pct)

full_controls['Total population'] = full_controls['Total population'].apply(clean_totals)
full_controls['Total housing units'] = full_controls['Total housing units'].apply(clean_totals)

full_controls['Density per unit'] = full_controls['Total population'] / full_controls['Total housing units']

full_controls['ZIP'] = full_controls['ZIP'].astype('int64')


# Creating Final Panel

In [20]:
#merge zoning df with rent df
temp_full = zip_panel.merge(changes, left_index = True, right_on = 'ZIP').set_index('ZIP')

In [37]:
#merge zoning+rent df with demographic controls
final_panel = (
    temp_full.merge(full_controls, 
                    left_index = True, right_on = 'ZIP')
    .set_index('ZIP')
)

final_panel['Median age (years)'] = final_panel['Median age (years)'].astype(float)

In [67]:
#log scaling columns
final_panel['log_0BR_base'] = np.log(final_panel['0BR_base'])
final_panel['log_1BR_base'] = np.log(final_panel['1BR_base'])
final_panel['log_2BR_base'] = np.log(final_panel['2BR_base'])
final_panel['log_3BR_base'] = np.log(final_panel['3BR_base'])
final_panel['log_4BR_base'] = np.log(final_panel['4BR_base'])

final_panel['log_population'] = np.log(final_panel['Total population'])

final_panel['log_housing_units'] = np.log(final_panel['Total housing units'])

final_panel['log_med_income'] = np.log(final_panel['Median income (dollars)'])

final_panel.replace(float('-inf'), np.NaN, inplace = True)

In [56]:
#doesn't work, area calculation incorrect
# final_panel['Population density (per km)'] = (
#     final_panel['Total population'] / (final_panel['total_area'] / 10**6)
# )

In [57]:
#sj_zips[sj_zips['ZIPCODE'] == '95014']['geometry'].plot()

In [68]:
final_panel.to_csv('full_data.csv')

# EDA

In [69]:
corr = final_panel.corr(numeric_only = False)
corr.style.background_gradient(cmap='coolwarm')

,multifam_percentage_2012,multifam_percentage_2015,multifam_percentage,initial_diff,final_diff,0BR_change,1BR_change,2BR_change,3BR_change,4BR_change,0BR_base,1BR_base,2BR_base,3BR_base,4BR_base,Male,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Hispanic or Latino (of any race),Total population,Median age (years),Total housing units,Median income (dollars),Mean income (dollars),Density per unit,log_0BR_base,log_1BR_base,log_2BR_base,log_3BR_base,log_4BR_base,log_population,log_units,log_med_income,log_housing_units
multifam_percentage_2012,1.000000,0.998048,0.715344,0.200526,-0.128653,-0.018428,0.041214,0.017891,0.135439,0.073207,-0.487072,-0.492313,-0.489596,-0.490722,-0.490339,0.049382,0.134743,-0.025150,0.238672,-0.195304,-0.251798,0.037081,0.320728,0.014022,0.402179,-0.306445,-0.217615,-0.232459,-0.491472,-0.496643,-0.494266,-0.495525,-0.494964,0.238499,0.299242,-0.305776,0.299242
multifam_percentage_2015,0.998048,1.000000,0.707348,0.261321,-0.142060,-0.031244,0.025528,0.003151,0.115705,0.055869,-0.463641,-0.468679,-0.466148,-0.467319,-0.466918,0.041428,0.133738,-0.056598,0.212976,-0.189321,-0.247400,0.036293,0.331788,0.006198,0.416571,-0.284996,-0.200633,-0.216156,-0.466810,-0.471733,-0.469590,-0.470914,-0.470328,0.259258,0.316336,-0.274944,0.316336
multifam_percentage,0.715344,0.707348,1.000000,0.039927,0.599210,0.143892,0.224250,0.197235,0.346331,0.271771,-0.638906,-0.641003,-0.638231,-0.638286,-0.638372,0.212991,0.054108,0.244978,0.264175,-0.278002,-0.337014,0.292006,0.029384,-0.167955,0.128472,-0.576023,-0.466458,-0.407618,-0.676369,-0.678799,-0.676390,-0.676330,-0.676583,-0.120890,-0.036845,-0.660117,-0.036845
initial_diff,0.200526,0.261321,0.039927,1.000000,-0.240059,-0.205299,-0.236536,-0.227088,-0.278241,-0.255034,0.254955,0.256938,0.254655,0.253669,0.254038,-0.110288,0.016296,-0.487880,-0.337675,0.045747,0.008164,-0.003351,0.247630,-0.116606,0.318715,0.256628,0.209069,0.195175,0.273261,0.275955,0.272820,0.271516,0.272035,0.374257,0.332385,0.400583,0.332385
final_diff,-0.128653,-0.142060,0.599210,-0.240059,1.000000,0.236887,0.285116,0.272629,0.353937,0.317297,-0.369574,-0.366805,-0.365789,-0.364540,-0.365115,0.248614,-0.073204,0.401324,0.129523,-0.174819,-0.192127,0.363536,-0.334634,-0.239060,-0.291900,-0.480953,-0.422843,-0.324316,-0.418446,-0.416274,-0.415326,-0.413743,-0.414761,-0.454055,-0.401031,-0.608339,-0.401031
0BR_change,-0.018428,-0.031244,0.143892,-0.205299,0.236887,1.000000,0.991279,0.993985,0.938682,0.971984,0.072076,0.075440,0.076684,0.077583,0.078212,0.025892,-0.117895,0.193499,-0.269554,0.263920,-0.403357,-0.506226,-0.278356,0.459120,-0.205075,0.261298,0.359256,-0.488986,0.046013,0.048142,0.049952,0.051256,0.051440,-0.242992,-0.155015,0.063427,-0.155015
1BR_change,0.041214,0.025528,0.224250,-0.236536,0.285116,0.991279,1.000000,0.998974,0.975646,0.993976,-0.054462,-0.051224,-0.049984,-0.048992,-0.048455,0.071344,-0.082327,0.203912,-0.223910,0.201417,-0.445909,-0.445570,-0.285116,0.438340,-0.210496,0.187292,0.288893,-0.508939,-0.080739,-0.078658,-0.076945,-0.075569,-0.075461,-0.271645,-0.179891,-0.019732,-0.179891
2BR_change,0.017891,0.003151,0.197235,-0.227088,0.272629,0.993985,0.998974,1.000000,0.968513,0.991251,-0.019134,-0.015693,-0.014613,-0.013614,-0.013064,0.065388,-0.091811,0.188504,-0.242621,0.218154,-0.443286,-0.454597,-0.284500,0.445893,-0.215047,0.217349,0.314092,-0.488868,-0.045584,-0.043314,-0.041753,-0.040366,-0.040249,-0.273662,-0.186120,0.009952,-0.186120
3BR_change,0.135439,0.115705,0.346331,-0.278241,0.353937,0.938682,0.975646,0.968513,1.000000,0.992277,-0.265798,-0.262452,-0.261456,-0.260457,-0.259971,0.142750,-0.020915,0.208179,-0.138741,0.088551,-0.497038,-0.319866,-0.282749,0.386136,-0.211757,0.057050,0.159780,-0.511824,-0.290706,-0.288329,-0.287027,-0.285697,-0.285597,-0.303075,-0.210389,-0.155142,-0.210389
4BR_change,0.073207,0.055869,0.271771,-0.255034,0.317297,0.971984,0.993976,0.991251,0.992277,1.000000,

# 0br 

In [63]:
indep = ['multifam_percentage_2012', 'initial_diff', 'final_diff',
        'White',
       'Black or African American', 'American Indian and Alaska Native',
       'Asian', 'Native Hawaiian and Other Pacific Islander',
       'Hispanic or Latino (of any race)', 'Total population',
       'Median age (years)', 'Total housing units', 'Median income (dollars)',
       'Mean income (dollars)', 'Density per unit', 'log_0BR_base',
       'log_1BR_base', 'log_2BR_base', 'log_3BR_base', 'log_4BR_base',
       'log_population', 'log_units', 'log_med_income', 'log_housing_units']

In [71]:
for var in indep:

    print(f"The correlation between 0br_change and {var} is: {final_panel['0BR_change'].corr(final_panel[var])}")

The correlation between 0br_change and multifam_percentage_2012 is: -0.018428190217719843
The correlation between 0br_change and initial_diff is: -0.20529879124186193
The correlation between 0br_change and final_diff is: 0.2368866005896452
The correlation between 0br_change and White is: -0.11789479731978526
The correlation between 0br_change and Black or African American is: 0.19349938484426657
The correlation between 0br_change and American Indian and Alaska Native is: -0.2695543438476864
The correlation between 0br_change and Asian is: 0.26392033329725256
The correlation between 0br_change and Native Hawaiian and Other Pacific Islander is: -0.40335710480469816
The correlation between 0br_change and Hispanic or Latino (of any race) is: -0.5062260420345247
The correlation between 0br_change and Total population is: -0.2783560662077859
The correlation between 0br_change and Median age (years) is: 0.45912002510165933
The correlation between 0br_change and Total housing units is: -0.2050

# Rezoning correlations

In [82]:
final_panel.columns

Index(['multifam_percentage_2012', 'multifam_percentage_2015',
       'multifam_percentage', 'initial_diff', 'final_diff', '0BR_change',
       '1BR_change', '2BR_change', '3BR_change', '4BR_change', '0BR_base',
       '1BR_base', '2BR_base', '3BR_base', '4BR_base', 'Male', 'White',
       'Black or African American', 'American Indian and Alaska Native',
       'Asian', 'Native Hawaiian and Other Pacific Islander',
       'Hispanic or Latino (of any race)', 'Total population',
       'Median age (years)', 'Total housing units', 'Median income (dollars)',
       'Mean income (dollars)', 'Density per unit', 'log_0BR_base',
       'log_1BR_base', 'log_2BR_base', 'log_3BR_base', 'log_4BR_base',
       'log_population', 'log_units', 'log_med_income', 'log_housing_units'],
      dtype='object')

In [86]:
rezone_factors = ['multifam_percentage_2012', 'initial_diff',
        'White',
       'Black or African American', 'American Indian and Alaska Native',
       'Asian', 'Native Hawaiian and Other Pacific Islander',
       'Hispanic or Latino (of any race)', 'Total population',
       'Median age (years)', 'Total housing units', 'Median income (dollars)',
       'Mean income (dollars)', 'Density per unit', 'log_0BR_base',
       'log_1BR_base', 'log_2BR_base', 'log_3BR_base', 'log_4BR_base',
       'log_population', 'log_units', 'log_med_income', 'log_housing_units']

In [87]:
for var in rezone_factors:

    print(f"The correlation between final_diff and {var} is: {final_panel['final_diff'].corr(final_panel[var])}")

The correlation between final_diff and multifam_percentage_2012 is: -0.12865252996638146
The correlation between final_diff and initial_diff is: -0.24005881257221848
The correlation between final_diff and White is: -0.07320381808987696
The correlation between final_diff and Black or African American is: 0.40132361914535625
The correlation between final_diff and American Indian and Alaska Native is: 0.12952335040284413
The correlation between final_diff and Asian is: -0.17481920426595696
The correlation between final_diff and Native Hawaiian and Other Pacific Islander is: -0.1921265857770428
The correlation between final_diff and Hispanic or Latino (of any race) is: 0.36353637441066566
The correlation between final_diff and Total population is: -0.33463367611026273
The correlation between final_diff and Median age (years) is: -0.23905960076221172
The correlation between final_diff and Total housing units is: -0.2918997755552603
The correlation between final_diff and Median income (dolla

# Imports for model

In [78]:
from pystata import config
from pystata.crunch import Crunch

# Set your Stata executable path
config.init_stata("Macintosh HD/Applications/Stata/StataSE")
crunch = Crunch()

ModuleNotFoundError: No module named 'pystata'

# Rent model

regress br_change final_diff initial_diff blackorafricanamerican hispanicorlatinoofanyrace log_0br_base medianageyears medianincomedollars densityperunit log_housing_units


# Rezoning model